In [124]:
# in case there are any problems with importing because path is wrong
import sys
sys.path.append('/Users/daniel/Princeton Dropbox/Daniel Gurevich/Research/discrete_sr/code/SPIDER_discrete')

import numpy as np
from sparse_reg_bf import *
from sparse_reg import regress

In [83]:
w = 100
h = 150
v = np.zeros([w, 1])
k = 10
v[0:10, 0] = [1, 1, 1, 1, 1, 1, 0.8, 0.6, 0.4, 0.2]

def generate_random_Q(inv_snr):
    # inv_snr: (signal to noise ratio)^-1
    U = np.random.uniform(-0.5, 0.5, [h, w])
    W = np.random.uniform(0, 1, [h, w])
    #U = np.random.uniform(0, 1, [w, h])
    
    #Sigma = (U.T @ U)/p + snr/(np.linalg.norm(v)**2)*(v @ v.T)
    
    true_matrix = W - (W @ v @ v.T)/(np.linalg.norm(v)**2)
    inv_row_norms = np.reshape(1/np.linalg.norm(true_matrix, axis=1), [h, 1])
    true_matrix = true_matrix * inv_row_norms # rescale row norms to 1
    Q = true_matrix+U*inv_snr/np.sqrt(w)
    return Q

def assess_Q(Q):
    true_residual = np.linalg.norm(Q @ v)
    _, _, V = np.linalg.svd(Q)
    best_sv = V[-1, :]/V[-1, 0]
    return true_residual, best_sv

Q = generate_random_Q(sigma)
true_residual, best_sv = assess_Q(Q)
print("Q:", Q)
print("Residual with true solution:", true_residual)
print("Best singular vector:", best_sv)

Q: [[-0.034  0.092 -0.024 ...  0.174  0.167  0.017]
 [ 0.027  0.013 -0.087 ...  0.11   0.146  0.142]
 [-0.017 -0.022  0.014 ...  0.08   0.081  0.056]
 ...
 [-0.031 -0.006 -0.04  ...  0.022  0.12   0.004]
 [-0.059 -0.009  0.03  ...  0.174  0.103  0.053]
 [-0.028 -0.072  0.016 ...  0.009  0.125  0.041]]
Residual with true solution: 0.009604032879897523
Best singular vector: [ 1.000e+00  1.000e+00  1.003e+00  9.998e-01  1.000e+00  1.000e+00
  7.988e-01  6.006e-01  3.971e-01  2.013e-01 -1.486e-03  5.206e-03
  3.136e-03  4.238e-03  5.275e-03  2.153e-03  2.859e-03  1.362e-03
 -2.434e-03  3.542e-03  2.220e-04 -3.721e-04 -6.162e-04 -2.746e-03
  4.336e-04 -1.352e-03 -1.431e-03 -4.018e-04 -6.865e-05 -4.782e-04
  2.545e-03 -1.448e-03  1.096e-03  5.276e-04 -3.762e-03  1.416e-03
 -4.559e-03 -4.623e-04  8.143e-04  9.160e-04 -5.735e-04  9.046e-05
 -3.554e-03 -4.677e-05 -1.387e-03 -6.996e-04 -1.522e-03  4.358e-03
 -2.961e-03 -1.580e-03  2.530e-03  3.916e-04  2.281e-03  2.190e-03
 -5.092e-03 -1.027e-03

In [138]:
%%prun

sigma = 1/3 # (signal to noise ratio)^-1
N_trials = 100

verbose = False

pr_vec = []
sr_vec = []
for i in range(N_trials):
    print("i: ", i)
    scaler = Scaler(sub_inds=None, char_sizes=np.ones([w]), row_norms=None, unit_rows=True)
    #init = Initializer(method='combinatorial', start_k=2)
    #init = Initializer(method='combinatorial', start_k=9999)
    init = Initializer(method='power', start_k=20)
    res = Residual(residual_type='matrix_relative')
    iterator = ModelIterator(max_k=20, backward_forward=True, max_passes=10)
    thres = Threshold(threshold_type=None, n_terms=10)

    Q = generate_random_Q(sigma)
    true_residual, best_sv = assess_Q(Q)
    reg_result = sparse_reg_bf(Q, scaler, init, res, iterator, thres, verbose=verbose)
    xi2, _ = regress(Q, list(range(k)), np.ones([w]))
    
    achieved_residual = np.linalg.norm(Q @ reg_result.xi)
    adj_true_residual = np.linalg.norm(Q @ xi2)
    performance_ratio = achieved_residual/adj_true_residual
    support_recovery = sum(abs(reg_result.xi[0:k])>0)/k
    pr_vec.append(performance_ratio)
    sr_vec.append(support_recovery)
    #print("Recovered coefficient vector:", reg_result.xi)
    #print("Recovered support:", np.where(abs(reg_result.xi>0)))
    #print("Ratio of achieved residual to residual achieved by true solution: ", performance_ratio)
    #print("Fraction of correct support recovered: ", support_recovery)

print("Mean performance ratio:", np.round(np.mean(pr_vec), 3))
print("Mean support recovery:", np.round(np.mean(sr_vec), 3))

i:  0
i:  1
i:  2
i:  3
i:  4
i:  5
i:  6
i:  7
i:  8
i:  9
i:  10
i:  11
i:  12
i:  13
i:  14
i:  15
i:  16
i:  17
i:  18
i:  19
i:  20
i:  21
i:  22
i:  23
i:  24
i:  25
i:  26
i:  27
i:  28
i:  29
i:  30
i:  31
i:  32
i:  33
i:  34
i:  35
i:  36
i:  37
i:  38
i:  39
i:  40
i:  41
i:  42
i:  43
i:  44
i:  45
i:  46
i:  47
i:  48
i:  49
i:  50
i:  51
i:  52
i:  53
i:  54
i:  55
i:  56
i:  57
i:  58
i:  59
i:  60
i:  61
i:  62
i:  63
i:  64
i:  65
i:  66
i:  67
i:  68
i:  69
i:  70
i:  71
i:  72
i:  73
i:  74
i:  75
i:  76
i:  77
i:  78
i:  79
i:  80
i:  81
i:  82
i:  83
i:  84
i:  85
i:  86
i:  87
i:  88
i:  89
i:  90
i:  91
i:  92
i:  93
i:  94
i:  95
i:  96
i:  97
i:  98
i:  99
Mean performance ratio: 1.076
Mean support recovery: 0.946
 

         33065268 function calls (33063055 primitive calls) in 52.578 seconds

   Ordered by: internal time

   ncalls  tottime  percall  cumtime  percall filename:lineno(function)
   957857   40.467    0.000   44.129    0.000 _linalg.py:1613(svd)
   957657    3.439    0.000   47.650    0.000 sr_utils.py:9(smallest_sv)
     9291    2.024    0.000   42.999    0.005 sparse_reg_bf.py:285(pick)
     5100    1.079    0.000    1.126    0.000 _linalg.py:1303(eig)
   968057    0.489    0.000    0.936    0.000 _linalg.py:148(_commonType)
  2889071    0.387    0.000    0.387    0.000 {method 'astype' of 'numpy.ndarray' objects}
   968057    0.332    0.000    0.648    0.000 _ufunc_config.py:410(__enter__)
  1936014    0.290    0.000    0.439    0.000 _linalg.py:142(_realType)
     9348    0.283    0.000    6.199    0.001 sparse_reg_bf.py:257(drop)
   973057    0.269    0.000    0.434    0.000 _linalg.py:123(_makearray)
  2020753    0.231    0.000    0.339    0.000 _linalg.py:128(isComplexType)
  

In [139]:
print(best_sv)
print(reg_result.xi)

[ 1.     0.966  0.921  0.899  0.905  1.028  0.796  0.486  0.252  0.278
 -0.052 -0.032 -0.012 -0.055  0.003 -0.09   0.041  0.109  0.136  0.002
  0.052  0.023 -0.014 -0.142 -0.075  0.014  0.036 -0.091  0.067 -0.018
 -0.094 -0.066  0.033  0.024  0.042 -0.17   0.05  -0.013 -0.115  0.011
  0.042 -0.012  0.002  0.026 -0.071 -0.003  0.061 -0.131 -0.082  0.096
 -0.102 -0.157  0.023  0.03   0.056 -0.06   0.103 -0.027  0.055  0.108
 -0.034 -0.043 -0.035  0.004  0.037 -0.156  0.05   0.032  0.101 -0.025
  0.021  0.027  0.003 -0.059 -0.024  0.118  0.034 -0.004  0.179 -0.019
  0.017  0.117  0.084 -0.011  0.067  0.002  0.135 -0.023  0.095  0.013
 -0.019  0.035 -0.174 -0.1   -0.054  0.044  0.008  0.038 -0.048 -0.024]
[0.983 0.989 1.    0.977 0.983 0.986 0.884 0.523 0.32  0.245 0.    0.
 0.    0.    0.    0.    0.    0.    0.    0.    0.    0.    0.    0.
 0.    0.    0.    0.    0.    0.    0.    0.    0.    0.    0.    0.
 0.    0.    0.    0.    0.    0.    0.    0.    0.    0.    0.    0.
 0.    0.

In [ ]:
SNR = 3

start_k=9999, no bf: (245 sec)
Mean performance ratio: 1.001
Mean support recovery: 0.994

start_k=2, no bf: (22 sec)
Mean performance ratio: 1.305
Mean support recovery: 0.815

start_k=2, 2 bf passes:
Mean performance ratio: 1.266
Mean support recovery: 0.865

start_k=2, 10 bf passes: (65 sec)
Mean performance ratio: 1.158
Mean support recovery: 0.895

power (k=20), no bf: (3 sec)
Mean performance ratio: 2.078
Mean support recovery: 0.567

power (k=20), 10 bf passes: (55 sec)
Mean performance ratio: 1.076
Mean support recovery: 0.946

In [ ]:
SNR = 2 (most PRs compared to the true solution, not adjusted solution on true support!)

start_k=9999, no bf: (255 sec)
Mean performance ratio: 0.892 / 0.891 (actually 0.997)
Mean support recovery: 0.954 / 0.971

start_k=2, no bf:
Mean performance ratio: 1.072
Mean support recovery: 0.74

start_k=2, 2 bf passes:
Mean performance ratio: 1.065
Mean support recovery: 0.75 / 0.793

start_k=2, 10 bf passes: (65 sec)
Mean performance ratio: 1.009 / 0.983
Mean support recovery: 0.847 / 0.841
 
power (k=20), no bf:
Mean performance ratio: 1.684
Mean support recovery: 0.328

power (k=40), no bf:
Mean performance ratio: 1.651
Mean support recovery: 0.319

power (k=20), 2 bf passes:
Mean performance ratio: 1.118
Mean support recovery: 0.723

power (k=20), 10 bf passes: (56 sec)
Mean performance ratio: 0.944 / 0.977
Mean support recovery: 0.898 / 0.848

power (k=20), 50 bf passes: (263 sec)
Mean performance ratio: 0.975
Mean support recovery: 0.864

power (k=40), 10 bf passes: (226 sec)
Mean performance ratio: 0.963
Mean support recovery: 0.857